### This notebook import images associated with each nucleus, correct for bleedthrough and compute NFRET pixel by pixel. Distribution of average NFRET value are computed over all nucleus in a biological replicate and a label low-mid-high is attributed to each nucleus. 

In [2]:

import pandas as pd
import numpy as np
from skimage import io
import pandas as pd
import os
import numpy as np
import pandas as pd
import pickle


#General Variables 

#bleedthrough values used for correction 

bt_D=0.67 # signal bleed-through (SBT) of the donor in the SE channel
bt_A=0.84 #SBT of the acceptor in the SE channel
bt_RCamp=0.26 #SBT of the RCamp in the SE channel


In [3]:
# Path to the directory containing FRET files output from FIJI (position of each ROI in the original image and classification from QuPath)
path_fret = r'C:\Users\SimonZ\Desktop\Code\graffproject\FRETRec\fijioutput'

# Path to the directory containing TIFF files for each cell
path_to_tiffs= r'C:\Users\SimonZ\Desktop\Code\graffproject\FRETRec\rois'

# List to store paths of FRET files
list_of_fret_files = []

# Walk through the directory structure to find FRET files
for root, dirs, files in os.walk(path_fret):
    for file in files:
        # Get the full path of the file
        file_path = os.path.join(root, file)
        
        # Append the file path to the list
        list_of_fret_files.append(file_path)

# Print the paths of FRET files
for file_path in list_of_fret_files:
    print(file_path)

C:\Users\SimonZ\Desktop\Code\graffproject\FRETRec\fijioutput\01.06_H3_Well3.csv
C:\Users\SimonZ\Desktop\Code\graffproject\FRETRec\fijioutput\31.05_H3_Well5.csv
C:\Users\SimonZ\Desktop\Code\graffproject\FRETRec\fijioutput\31.05_H3_Well6.csv


In [4]:
# List to store FRET cell data
fret_cells = []

# Iterate over each file (one file by recording, output from FIJI)
for file_path in list_of_fret_files:
    # Extract the name of the file
    name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Read the data from the CSV file
    data = pd.read_csv(file_path)

    nb_frame = data.max()['Frame']
    nb_channel = data.max()['Ch']
    nb_cells = int(len(data)/(nb_frame*nb_channel))

    # Iterate over each cell in the data
    for i in range(nb_cells):
        
        # Check if the cell is classified as positive by QuPath
        if 'Positive' in data.iloc[i]["Label"]:     
            # Extract X and Y coordinates of the cell
            x_y_coordinates = np.array([data.iloc[i]['X'], data.iloc[i]['Y']])
            
            # Create a dictionary containing cell information
            cell_data = {
                'Label': name + '_' + str(i), # Label attributed by FIJI
                'ROI_Number': data.iloc[i].iloc[0], # Number of the ROI tiff file
                "Coordinates": x_y_coordinates  # X and Y coordinates of the cell
            }
            
            # Append the cell data to the list of FRET data
            fret_cells.append(cell_data)

# Output the extracted FRET data
for fret_data in fret_cells:
    print(fret_data)

{'Label': '01.06_H3_Well3_1', 'ROI_Number': 2, 'Coordinates': array([409.535, 500.357])}
{'Label': '01.06_H3_Well3_2', 'ROI_Number': 3, 'Coordinates': array([409.133, 401.363])}
{'Label': '01.06_H3_Well3_3', 'ROI_Number': 4, 'Coordinates': array([571.123, 351.699])}
{'Label': '01.06_H3_Well3_4', 'ROI_Number': 5, 'Coordinates': array([ 41.712, 365.882])}
{'Label': '01.06_H3_Well3_5', 'ROI_Number': 6, 'Coordinates': array([516.186, 401.747])}
{'Label': '01.06_H3_Well3_6', 'ROI_Number': 7, 'Coordinates': array([490.481, 395.921])}
{'Label': '01.06_H3_Well3_8', 'ROI_Number': 9, 'Coordinates': array([425.105, 148.616])}
{'Label': '01.06_H3_Well3_9', 'ROI_Number': 10, 'Coordinates': array([308.069, 192.033])}
{'Label': '01.06_H3_Well3_10', 'ROI_Number': 11, 'Coordinates': array([616.264, 395.313])}
{'Label': '01.06_H3_Well3_11', 'ROI_Number': 12, 'Coordinates': array([320.827, 186.102])}
{'Label': '01.06_H3_Well3_12', 'ROI_Number': 13, 'Coordinates': array([148.514, 374.905])}
{'Label': '01.

In [6]:
def return_tiff(cell, path_to_tiffs):
    """
    Return the TIFF image data for a given cell.

    Parameters:
        cell (dict): A dictionary containing information about the cell.
        path_to_tiffs (str): The path to the directory containing TIFF files.

    Returns:
        numpy.ndarray: The TIFF image data for the specified cell.
    """
    # Split the cell label to extract recording name and ROI number
    label_parts = cell["Label"].split('_')
    recording_name = '_'.join(label_parts[:3])  # Extract recording name
    roi_number = cell["ROI_Number"]  # Extract ROI number
    
    # Construct the path to the TIFF file using recording name and ROI number
    tiff_path = f"{path_to_tiffs}/{recording_name}/{recording_name}_ROI{roi_number}.tif"
    
    # Read the TIFF file
    tiff = io.imread(tiff_path)

    # fig, ax = plt.subplots(nrows=1, ncols=6,figsize=(5, 10)) #to plot in case
    # ax[0].imshow(tiff[0,:,:]) #Exc-Donor and Em-Donor 
    # ax[1].imshow(tiff[1,:,:]) #Exc-Donor and Em-Acceptor 
    # ax[2].imshow(tiff[2,:,:]) # Transmitted light
    # ax[3].imshow(tiff[3,:,:]) #Exc-Acceptor and Em-Acceptor 
    # ax[4].imshow(tiff[4,:,:]) #Exc-RCamp and Em-RCamp 
    # ax[5].imshow(tiff[6,:,:]) # Each pixel has the value of the number of the ROI

    # # Return the TIFF data
    return tiff

In [7]:
def build_cell_pxl_info(cell, path_to_tiffs, channel_D=0, channel_SE=1, channel_A=3, channel_RCamp=4):
    """
    Build information about pixels associated with a cell's ROI from a TIFF file.

    Parameters:
        cell (dict): A dictionary containing information about the cell.
        path_to_tiffs (str): The path to the directory containing TIFF files.
        channel_D (int): Channel index for intensity in the donor channel.
        channel_SE (int): Channel index for intensity in the FRET channel.
        channel_A (int): Channel index for intensity in the acceptor channel.
        channel_RCamp (int): Channel index for intensity in the RCamp channel.

    Returns:
        pd.DataFrame: DataFrame containing pixel information for the cell's ROI.
    """
    # Retrieve TIFF data for the cell
    tiff_file = return_tiff(cell, path_to_tiffs)
    nb_roi = cell["ROI_Number"]
    
    # Lists to store pixel information for each channel
    I_donor = []  # Exc-Donor and Em-Donor
    I_SE = []  # Exc-Donor and Em-Acceptor
    I_acceptor = []  # Exc-Acceptor and Em-Acceptor
    I_RCamp = []  # Exc-RCamp and Em-RCamp

    I_outside_ROI_acceptor = [] #store value of the pixel outside the ROI to compute the background value 
    dim = tiff_file.shape

    tot_area = 0
    area_with_signal = 0
    
    # Iterate over all pixels in the TIFF file
    for i in range(dim[1]):
        for j in range(dim[2]):
            # Check if the pixel is part of the ROI
            if tiff_file[-1, i, j] == nb_roi:
                tot_area += 1
                
                # Check if the pixel has values for all channels
                if (not np.isnan(tiff_file[channel_D, i, j])) and \
                   (not np.isnan(tiff_file[channel_SE, i, j])) and \
                   (not np.isnan(tiff_file[channel_A, i, j])) and \
                   (not np.isnan(tiff_file[channel_RCamp, i, j])):

                    # Store pixel values for each channel
                    I_donor.append(tiff_file[channel_D, i, j] + 1)
                    I_SE.append(tiff_file[channel_SE, i, j] + 1)
                    I_acceptor.append(tiff_file[channel_A, i, j] + 1)
                    I_RCamp.append(tiff_file[channel_RCamp, i, j] + 1)
                    area_with_signal += 1
            else:
                #If not in the ROI, store value of the acceptor channel
                I_outside_ROI_acceptor.append(tiff_file[channel_A, i, j] + 1)

    # Create a DataFrame with pixel information
    df = {'donor': I_donor, 'SE': I_SE, 'acceptor': I_acceptor, 'RCamp': I_RCamp, 'ROI_area': tot_area, 'area_with_signal': area_with_signal,'background_acceptor':np.mean(I_outside_ROI_acceptor)}
    ROI_info = pd.DataFrame(df)
    return ROI_info


In [8]:
def compute_NFRET(cell, bt_D_D, bt_A_A, bt_RCamp):
    """
    Compute the normalized FRET (NFRET) value for a given cell.

    Parameters:
        cell (dict): A dictionary containing information about the cell.
        bt_D_D (float): Bleedthrough value for the donor channel.
        bt_A_A (float): Bleedthrough value for the acceptor channel.
        bt_RCamp (float): Bleedthrough value for the RCamp channel.

    Returns:
        float: The computed normalized FRET (NFRET) value.
    """
    # Extract pixel intensities from the cell data
    I_donor = cell["Pixelval"]["donor"]     # Pixel intensity of donor channel
    I_SE = cell["Pixelval"]["SE"]           # Pixel intensity of sensitize emission (SE) channel
    I_acceptor = cell["Pixelval"]["acceptor"]   # Pixel intensity of acceptor channel
    I_RCamp = cell["Pixelval"]["RCamp"]     # Pixel intensity of RCamp channel
    
    # Compute normalized FRET (NFRET) using the provided bleedthrough values
    NFRET = (I_SE - I_donor * bt_D_D - I_acceptor * bt_A_A - I_RCamp * bt_RCamp) / I_acceptor
    
    return NFRET


In [9]:
def compute_mean_intensity(cell):
    """
    Compute the mean intensity values for each channel and the mean NFRET value for a given cell.

    Parameters:
        cell (dict): A dictionary containing information about the cell.

    Returns:
        dict: A dictionary containing the mean intensity values for each channel and the mean NFRET value.
    """
    # Extract pixel intensities and NFRET value from the cell data
    I_D = cell["Pixelval"]["donor"]      # All ROI pixel intensity of donor channel
    I_SE = cell["Pixelval"]["SE"]        # All ROI pixel intensity of ensitize emission (SE) channel
    I_A = cell["Pixelval"]["acceptor"]   # All ROI pixel intensity of acceptor channel
    I_RCamp = cell["Pixelval"]["RCamp"]  # All ROI pixel intensity of RCamp channel
    NFRET = cell["Pixelval"]["NFRET"]    # All ROI pixel intensity of normalized FRET (NFRET) value
    
    # Compute the mean intensity values for each channel and the mean NFRET value
    mean_intensity = {
        "donor": np.mean(I_D),
        "SE": np.mean(I_SE),
        "acceptor": np.mean(I_A),
        "RCamp": np.mean(I_RCamp),
        "NFRET": np.mean(NFRET)
    }
    
    return mean_intensity

# Extract the information from the Tiff file

In [10]:
# Compute the NFRET for each cell
for cell in fret_cells:
    cell["Pixelval"]= build_cell_pxl_info(cell, path_to_tiffs)
    cell["Pixelval"]["NFRET"]= compute_NFRET(cell, bt_D, bt_A, bt_RCamp)
    cell["meanval"] = compute_mean_intensity(cell)

### We exclude cell showing saturation in more than 10% of their pixel and/or showing unsufficient signal in either I_D or I_A

In [11]:
def sufficient_signal(cell, contrast_value_I_A):
    """
    Check if the signal in a cell is sufficient for analysis based on given criteria.

    Parameters:
        cell (dict): A dictionary containing information about the cell.
        contrast_value_I_A (float): Contrast value for the acceptor channel.

    Returns:
        bool: True if the signal is sufficient, False otherwise.
    """
    # Initialize flag for signal sufficiency
    sufficient_signal = True
    
    # Check conditions for signal sufficiency
    if contrast_value_I_A < 1.8 and cell["meanval"]["acceptor"] < 20:  # Low contrast and low intensity in acceptor channel  -> This is a nucleus without signal in the acceptor channel
        sufficient_signal = False
    if contrast_value_I_A >= 1.8 and cell["meanval"]["acceptor"] < 8:  # High contrast but low intensity in acceptor channel -> This is a nucleus without signal in the acceptor channel
        sufficient_signal = False
    if cell["meanval"]["donor"] < 20 and cell["meanval"]["NFRET"] < 2:  # Low intensity in donor and NFRET channels -> This is a nucleus without signal in the donor or NFRET probably a false positive
        sufficient_signal = False
    
    return sufficient_signal


In [12]:
good_cells = []

for cell in fret_cells:
    #Check if the number of pixel with signal (non-saturated pixel) in a given ROI is at least 90% of the total area of the ROI
    if cell["Pixelval"]["area_with_signal"][0] > 0.9 * cell["Pixelval"]["ROI_area"][0]:
        contrast_value_I_A = cell["meanval"]["acceptor"] / np.mean(cell["Pixelval"]["background_acceptor"])

        if sufficient_signal(cell, contrast_value_I_A):
            good_cells.append(cell)

print("Total number of cells:", len(fret_cells))
print("Total number of cells showing good signal:", len(good_cells))


Total number of cells: 246
Total number of cells showing good signal: 125


## Attribute low-mid-high class depending on the NFRET value

In [13]:
# List to store NFRET values of good cells
NFRET_values = []

# Iterate over good cells to collect NFRET values
for cell in good_cells:
    NFRET_values.append(cell["meanval"]["NFRET"])

# Calculate mean and standard deviation of NFRET values
mean_NFRET = np.mean(NFRET_values)
std_FRET = np.std(NFRET_values)

# Iterate over good cells to classify them based on NFRET values
for cell in good_cells:
    # Initialize cell class as 'mid'
    cell["class"] = 'Mid'
    
    # Check if NFRET value is below the low threshold
    if cell["meanval"]["NFRET"] < mean_NFRET - std_FRET / 2:
        cell["class"] = 'Low'
    
    # Check if NFRET value is above the high threshold
    if cell["meanval"]["NFRET"] >= mean_NFRET + std_FRET / 2:
        cell["class"] = 'High'


## Save the result in a pickle file to use in the 2_CombineFRETCA.ipynb jupyter notebook for further analysis

In [12]:
# Save to pickle file
with open('FRETgood_cells.pkl', 'wb') as pickle_file:
    pickle.dump(good_cells, pickle_file)